In [1]:
%%capture
%run ch04_linear_algebra.ipynb
%run ch05_statistics.ipynb
%run ch08_gradient_descent.ipynb

In [2]:
def predict(alpha: float, beta: float, x_i: float) -> float:
    return beta * x_i + alpha

In [3]:
def error(alpha: float, beta: float, x_i: float, y_i: float) -> float:
    return predict(alpha, beta, x_i) - y_i

In [4]:
# Sum of the squared errors

def sum_of_sqerrors(alpha: float, beta: float, x: Vector, y: Vector) -> float:
    return sum(error(alpha, beta, x_i, y_i) ** 2 
               for x_i, y_i in zip(x, y))

In [5]:
# Least squares (finding alpha and beta that make the sum of the squared errors as small as possible)

from typing import Tuple

def least_squares_fit(x: Vector, y: Vector) -> Tuple[float, float]:
    """
    Given two vectors x and y,
    find the least-squares values of alpha and beta
    """
    
    beta = correlation(x, y) * standard_deviation(y) / standard_deviation(x)
    alpha = mean(y) - beta * mean(x)
    
    return alpha, beta

In [6]:
x = [i for i in range(-100, 110, 10)]
y = [3 * i - 5 for i in x] # y = 3x - 5

assert least_squares_fit(x, y) == (-5, 3)

In [7]:
alpha, beta = least_squares_fit(num_friends_good, daily_minutes_good)

assert 22.9 < alpha < 23.0
assert 0.9 < beta < 0.905

In [8]:
# R-squared (measuring the fraction of the total variation in the dependent variable that is captured by the model)

def total_sum_of_squares(y: Vector) -> float:
    """the total squared variation of y_i's from their mean"""
    
    return sum(v ** 2 for v in de_mean(y))

def r_squared(alpha: float, beta: float, x: Vector, y: Vector) -> float:
    """
    the fraction of variation in y captured by the model, which equals
    1 - the fraction of variation in y NOT captured by the model
    """
    
    return 1.0 - (sum_of_sqerrors(alpha, beta, x, y) / total_sum_of_squares(y))

rsq = r_squared(alpha, beta, num_friends_good, daily_minutes_good)
assert 0.328 < rsq < 0.330

In [9]:
# Using gradient descent

import random
import tqdm

num_epochs = 10000
random.seed(0)

guess = [random.random(), random.random()] # random initialization (y = bx + a)
learning_rate = 0.00001

with tqdm.trange(num_epochs) as t:
    for _ in t:
        alpha, beta = guess
        
        # Partial derivative of loss with respect to 'alpha'
        grad_a = sum(2 * error(alpha, beta, x_i, y_i) 
                     for x_i, y_i in zip(num_friends_good, daily_minutes_good))
        
        # Partial derivative of loss with respect to 'beta'
        grad_b = sum(2 * error(alpha, beta, x_i, y_i) * x_i 
                     for x_i, y_i in zip(num_friends_good, daily_minutes_good))
        
        # Compute loss to stick in the tqdm description
        loss = sum_of_sqerrors(alpha, beta, num_friends_good, daily_minutes_good)
        t.set_description(f"loss: {loss:.3f}")
        
        # Finally, update the guess
        guess = gradient_step(guess, [grad_a, grad_b], -learning_rate)

alpha, beta = guess
assert 22.9 < alpha < 23.0
assert 0.9 < beta < 0.905

loss: 13196.619: 100%|██████████████████████████████████████████| 10000/10000 [00:20<00:00, 485.81it/s]
